In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [6]:
import os
import zipfile
import urllib.request as request
from lone_approval_prediction.constants import *
from lone_approval_prediction import logger
from lone_approval_prediction.utils.comman import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config["artifacts_root"]])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]
        
        create_directories([config["root_dir"]])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config["root_dir"],
            source_url = config["source_url"],
            local_data_file = config["local_data_file"],
            unzip_dir = config["unzip_dir"]
            
        )
        
        return data_ingestion_config
    

In [7]:
import zipfile
import pandas
import shutil
import os
import urllib.request as request
from pathlib import Path
from lone_approval_prediction import logging
from lone_approval_prediction.utils.comman import get_size

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        local_data_file_dir = os.path.dirname(self.config.local_data_file)
        if not os.path.exists(local_data_file_dir):
            os.makedirs(local_data_file_dir, exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded successfully with following info:\n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted all files at {unzip_path}")


In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

2025-02-01 19:43:01,512: INFO: comman: yaml file : config\config.yaml lodded successfully
2025-02-01 19:43:01,512: INFO: comman: yaml file : params.yaml lodded successfully
2025-02-01 19:43:01,512: INFO: comman: yaml file : schema.yaml lodded successfully
2025-02-01 19:43:01,512: INFO: comman: created directory at :artifacts
2025-02-01 19:43:01,518: INFO: comman: created directory at :artifacts/data_ingestion
2025-02-01 19:43:02,988: INFO: 4212319911: artifacts/data_ingestion/data.zip downloaded successfully with following info:
Connection: close
Content-Length: 82535
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d3990bb007a03f79f966d4d9a3089d613854b2ebc1bba0789824cb64bb919241"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A7AC:08EF:A6D7D:FC381:679E2BF7
Accept-Ranges: bytes
Date: Sat, 0